In [1]:
from preprocess_data.parse_data import ParseHandler
from preprocess_data.chunking import ChunkHandler
import os
import io
api_key = os.environ["OPENAI_API_KEY"]

In [2]:
parsehandler = ParseHandler.get_instance(api_key=api_key)

In [3]:
def pdf_to_stream(file_path):
    with open(file_path, 'rb') as pdf_file:
        pdf_content = pdf_file.read()
        pdf_stream = io.BytesIO(pdf_content)
    return pdf_stream

In [7]:
file_path = 'thuvienphapluat/law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf'
pdf_stream = pdf_to_stream(file_path)

In [8]:
image_base64s = parsehandler.pdf_to_images(file_stream=pdf_stream, pdf_path=file_path)

In [10]:
def parsing(file_stream, pdf_path):
    parsehandler = ParseHandler.get_instance(api_key=api_key)
    image_base64s = parsehandler.pdf_to_images(file_stream, pdf_path)
    pages, pdf_info, tables, equations = parsehandler.parse_pdf(image_base64s, pdf_path)
    return pages, pdf_info, tables, equations

In [11]:
pages, pdf_info, tables, equations = parsing(pdf_stream, file_path)

1
1
1
1
1
1
1
1
1
1
1


In [12]:
import json
data = {"pages": pages, "pdf_info": pdf_info, "tables": tables, "equations": equations}
with open("parsing_result/Salary_regulation(V).json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

In [1]:
import json
with open("parsing_result/Salary_regulation(V).json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

In [6]:
with open(f"parsing_result/tables.md", "a", encoding="utf-8") as file:
    for table in data["tables"]:
        file.write(table["content"] + "\n\n")

In [ ]:
with open(f"parsing_result/equations.md", "a", encoding="utf-8") as file:
    for equation in data["equations"]:
        file.write(equation["content"] + "\n\n")

In [12]:
pages, pdf_info, tables, equations = data["pages"], data["pdf_info"], data["tables"], data["equations"]

In [10]:
with open("parsing_result/Salary_regulation(V).md", "w", encoding="utf-8") as file:
    file.write(pdf_info["content"])

In [13]:
chunkhandler = ChunkHandler.get_instance(api_key)

In [14]:
chunk = []

In [15]:
articles = chunkhandler.create_article(pdf_info, pages)
len(articles)

17

In [16]:
forms = chunkhandler.create_form(pdf_info, pages)
len(forms)

0

In [17]:
tbls = chunkhandler.create_table(tables)
len(tbls)

8

In [18]:
eqts = chunkhandler.create_equation(equations)
len(eqts)

8

In [19]:
chunk.extend(articles)
chunk.extend(tbls)
chunk.extend(eqts)
chunk.extend(forms)
len(chunk)

33

In [20]:
with open("parsing/Salary_regulation(V)_chunk.json", "w", encoding="utf-8") as json_file:
    json.dump(chunk, json_file, indent=4, ensure_ascii=False)